# Парсер для выгрузки данных с КУАПа

Этот код позволяет пользователю совершить выгрузку данных с сайта https://kuap.ru/ за определенный период
и добавляет эти данные в указанный файл.

Парсер принимает на вход файл(.xls) и передает его столбец в выбранный файл(.xlsx).

Если возникает ошибка с импортом библиотеки, вводим команду ! pip install <module_name> --user

### Импорт библиотек и модулей

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd
import numpy as np
import glob
import win32com.client as win32

### Объявление переменных, содержащих название файлов для выгрузки

In [10]:
#Итерируемые списки - соответствие ссылок и названий
#Месячные данные
paths_monthly = ('loans', 'secs_act', 'funds_banks', 'loans_jl_net', 'loans_fl_brt',
                 'loans_jl_brt', 'loans_all_net', 'loans_all_brt','stocks_act', 'pifs', 'pifs101', 'other_assets', 'perc2rec',
                 'for', 'loro', 'loro_r', 'loro_n', 'customer', 'acct_broker', 'acct_all', 'deps_client_all', 'acct_deps_fl',
                 'acct_deps_jl', 'secs_pas', 'bonds_pas', 'bills_pas', 'certs_pas','perc2pay', 'capital', 'warranties', 
                 'collateral','collateral_goods', 'collateral_securities', 'collateral_metal', 'all_liabilities', 
                 'convers_turn', 'cash', 'corr_n_all', 'corr_n_cb', 'corr_n_rez', 'corr_n_nerez', 'rez_high_liq','cap', 'ucap',
                 'emiss', 'subord', 'rfund', 're', 'reval', 'n4', 'n7', 'pfi','opex', 'c26404', 'c26408', 'staff', 'c26411',
                 'roae', 'roaa', 'ci', 'nim', 'ca', 'net_assets', 'bills_act', 'liabilites', 'deps_mbk_all', 'deps_rez',
                 'deps_nerez', 'deps_cb', 'rez_mbk', 'loans_fl_net', 'loans_fl', 'loans_fl_over','rez_fl','loans_jl',
                 'loans_jl_over','rez_jl', 'loans_all_overdue', 'mortgage_all', 'mortgage_npl', 'loans_all_norm', 'mbk',
                 'mbk_cb', 'mbk_rez', 'mbk_nerez', 'deps_jl', 'deps_fl', 'acct_fl', 'acct_jl', 'n2', 'n3','cap123', 'n1',
                 'brt_assets', 'high_liq')
titles_monthly = ('Кредиты клиентам', 'Ценные бумаги', 'Средства банков', 'Кредиты юрлицам нетто', 'Кредиты физлицам брутто',
                  'Кредиты юрлицам брутто', 'Кредиты всем нетто','Кредиты всем брутто','Вложения в акции', 
                  'Вложения в ПИФ и ДУ(ф135)', 'Вложения в ПИФ и ДУ(ф101)', 'Прочие активы', 'Начисленные проценты', 
                  'Фонд обязательных резервов', 'Лоро-счета', 'Лоро-счета резиденетов', 'Лоро-счета нерезидентов', 
                  'Средства клиентов', 'Брокерские счета', 'Текущие средства', 'Срочные средства', 'Средства физлиц',
                  'Средства юрлиц', 'Выпущенные ценные бумаги', 'Вложения в облигации', 'Выпущенные векселя', 
                  'Депозитные сертификаты','Проценты к уплате', 'Собственные средства', 'Выданные гарантии', 
                  'Обеспечение по кредитам', 'Обеспечение имуществом','Обеспечение ценными бумагами', 
                  'Обеспечение драг металлами', 'Все обязательства', 'Оборот по конверсии', 'Остатки в кассе и банкоматах', 
                  'Средства на корр счетах', 'Средства на счете в ЦБ', 'Средства в банках РФ','Средства в иностр банках', 
                  'Резервы по корр счетам', 'Капитал банка(134)', 'Уставный капитал', 'Эмиссионный доход', 
                  'Субординированные займы', 'Резервный фонд', 'Нераспределенная прибыль', 'Переоценка имущества','Норматив Н4',
                  'Норматив Н7', 'Доход от расчетных срочн сдел', 'Операционные расходы', 'Расходы на рекламу', 
                  'Расходы на аудит', 'Расходы на персонал', 'Налоги уплаченные(кроме НП)', 'Рентабильность капитала(ROE)',
                  'Рентабильность активов(ROA)', 'Cost to Income','Чистая процентная маржа(NIM)', 
                  'Стоимость активов(Cost to Assets)', 'Активы банка (активы нетто)', 'Вложения в векселя', 'Баланс пассивы',
                  'Выданные МБК', 'МБК резидентам', 'МБК нерезидентам', 'Депозиты в ЦБ', 'Резервы по МБК', 
                  'Кредиты физлицам нетто', 'Кредиты ФЛ непросроч', 'Кредиты ФЛ просрочен', 'Резервы по кредитам ФЛ',
                  'Кредиты ЮЛ непросроч','Кредиты ЮЛ просроч','Резервы по кредитам ЮЛ', 'Кредиты всем просроченные', 
                  'Жилищные кредиты (все)', 'Жилищные кредиты (просроченные)', 'Кредиты всем непросроченные', 
                  'МБК привлеченные', 'МБК от ЦБ', 'МБК от резидентов', 'МБК от нерезидентов', 'Депозиты юрлиц', 
                  'Вклады физ лиц', 'Счета физлиц', 'Счета юрлиц', 'Норматив Н2', 'Норматив Н3','Капитал банка 123',
                  'Нормативы банка(H1)', 'Активы до вычета резервов', 'Высоколиквидные активы')
#Квартальные данные (месяцы 1, 4, 7, 10)
paths_quarterly = ('roe', 'net_profit', 'adj_net_profit', 'ytd_profit', 'interest', 'interest_le', 'interest_ind', 'nii', 
                   'fees', 'fees_docum', 'fees_fcy', 'nci', 'nci_rko', 'fx', 'fx_real', 'fx_reval', 'securities_income', 
                   'securities_real', 'securities_reval')
titles_quarterly = ('Финансовые показатели(roe)', 'Чистая прибыль', 'Скорректированная прибыль', 'Прибыль с начала года', 
                    'Процентные доходы', 'Процент про кредитам ЮЛ', 'Процент про кредитам ФЛ', 'Чистый процентный доход', 
                    'Комиссионные доходы', 'Доход по документарным оперциям', 'Доход по валютному контролю', 
                    'Чистый комиссионный доход', 'Чистый комиссионный доход по РКО', 'Доходы по ин валюте', 
                    'Реализованные вал доходы', 'Нереализованные вал доходы', 'Доходы по ценным бумагам', 
                    'Торговый доход по бумагам', 'Переоценка ценных бумаг')

### Загрузка файлов

In [11]:
# Папку загрузок Телеграмма удалить вручную 
def clear_folder(path):
    """Функция чистит директорию загрузок, так как для дальнейшего выполнения программы требуется, чтобы она была пустой.
    Принимает в качестве аргумента строку с путем к директории Загрузок, возвращает None.

    Параметры
    ----------
    path : str
        путь, позволяющий перейти в директорию загрузок пользователя
        
    Returns
    -------
    None
    """
    files = glob.glob(path+'\\*')
    for f in files:
        os.remove(f)

In [12]:
# Драйвер браузера устанавливается пользователем вручную 
path_to_driver = r'C:\Users\Nina\Laba\\chromedriver.exe'
path_to_kuap = 'https://kuap.ru/banks/ranks/'

def download_files(paths, year, month):
    """Функция выгружает excel-файлы по заданному пути. 
    Принимает в качестве аргумента список/кортеж, возвращает None.

    Параметры
    ----------
    paths : tuple/list
        Пути к необходимым файлам на сайте КУАПа
    year : list
        список из одного элемента - строка с годом, нужного для выгрузки
    month : list
        список из одного элемента - строка с месяцем, нужного для выгрузки
        
    Returns
    -------
    None
    """
    browser = webdriver.Chrome(executable_path=path_to_driver) # путь к хром драйверу
    for q in range(len(year)):
        for l in range(len(month)):
            browser.get(path_to_kuap + paths + '/na/0-0-0-10000-0-0-0-0-0-0/'\
                        + year[q] + '-' + month[l] + '-01/' + year[q] + '-' + month[l] + '-01/')
            button1_xpath = "//input[@class='btn btn-primary' and @value='Обновить данные']"
            button2_xpath = "//button[@class='btn btn-default' and @onclick='javascript:submit(this);']"
            click2 = browser.find_element(By.XPATH, button2_xpath);
            click2.click()
            time.sleep(3) 
    browser.close()

In [13]:
def rename(path):
    """Функция переименовывает выгруженные файлы. Это необходимо, так как по умолчанию названия файлов
    генерируются случайным образом. Принимает в качестве аргумента строку 
    с путем к директории Загрузок, возвращает None.

    Параметры
    ----------
    path : str
        строка с названием пути, позволяющих перейти в директорию загрузок пользователя

    Returns
    -------
    None
    """
    files = os.listdir(path)
    count = 1
    for file in files:
        os.rename(os.path.join(path, file), os.path.join(path, '1 ('+str(count)+').xls'))
        count += 1

### Добавление новых данных в общий файл

In [14]:
path_to_repository = r'C:\BANKS\\'

def merge_ex_files1(titles):
    """Функция объединяет два excel-файла и добавляет в файл из репозитория выгруженные данные. 
    Принимает в качестве аргумента список/кортеж, возвращает None.

    Параметры
    ----------
    titles : tuple/list
        Строки с названиями файлов в директории репозитория
        
    Returns
    -------
    None
    """
    df_merge = pd.read_excel(path_to_repository + titles + '.xlsx', header=0)
    path_to_file = r'C:\Users\Nina\Downloads\1 (1).xls'
    excel_app = win32.Dispatch('Excel.Application')
    wb = excel_app.Workbooks.open(path_to_file)
    excel_app.DisplayAlerts = False
    wb.Save()
    excel_app.quit()
    df = pd.read_excel(path_to_file, header=3)
    df = df.iloc[:,[1,2,3]]
    df_merge = pd.merge(df_merge, df, how="outer", on=["Рег. №", "Название банка"]) #включаем также значения, которые отсутствуют или не совпадают
    df_merge = df_merge.replace(to_replace=np.nan, value=0)
    df_merge.drop_duplicates()
    del df
    df_merge.sort_values(by="Рег. №", ascending=True, inplace=True)
    df_merge.to_excel(path_to_repository + titles +'.xlsx', index=False)

### Удаление скачанных файлов из Загрузок

In [15]:
def delete_files(path):
    """Функция удаляет скачанный файл из директории 
       (так как на его месте в директорию скачается другой файл)
    
    Параметры
    ----------
    path : str
        строка с названием пути, позволяющих перейти в директорию загрузок пользователя
        
    Returns
    -------
    None
    """
    files = os.listdir(path)
    count = 1
    for file in files:
        os.remove(os.path.join(path, '1 ('+str(count)+').xls'))
        count += 1

### Основной код 

In [16]:
#Год и месяц изменить на требуемые

year = ['2022']
month = ['01']
paths = paths_quarterly
titles = titles_quarterly
path = r'C:\Users\Nina\Downloads'

tic = time.perf_counter()
for k in range(len(paths)):
    clear_folder(path)
    download_files(paths[k], year, month)
    rename(path)
    merge_ex_files1(titles[k])
    delete_files(path)
    print(titles[k]+'   completed')
toc = time.perf_counter()
parsing_time = toc - tic
print(f"Парсинг занял {parsing_time // 60} минут")

Финансовые показатели(roe)   completed
Чистая прибыль   completed
Скорректированная прибыль   completed
Прибыль с начала года   completed
Процентные доходы   completed
Процент про кредитам ЮЛ   completed
Процент про кредитам ФЛ   completed
Чистый процентный доход   completed
Комиссионные доходы   completed
Доход по документарным оперциям   completed
Доход по валютному контролю   completed
Чистый комиссионный доход   completed
Чистый комиссионный доход по РКО   completed
Доходы по ин валюте   completed
Реализованные вал доходы   completed
Нереализованные вал доходы   completed
Доходы по ценным бумагам   completed
Торговый доход по бумагам   completed
Переоценка ценных бумаг   completed
Парсинг занял 4.0 минут
